In [2]:
# 설정
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Sequential 모델을 사용하는 경우
    * Sequential 모델은 각 레이어에 정확히 하나의 입력 텐서와 하나의 출력 텐서가 있는 일반 레이어 스택에 적합하다.

In [3]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation = 'relu', name = 'layer1'),
        layers.Dense(3, activation = 'relu', name = 'layer2'),
        layers.Dense(4, name = 'layer3')
    ]
)

# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

다음은 위와 동일한 코드이다.

In [4]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

#### Sequential 모델은 다음의 경우에 적합하지 않다.
    * 모델에 다중 입력 또는 다중 출력이 있다.
    * 레이어에 다중 입력 또는 다중 출력이 있다.
    * 레이어 공유를 해야 한다.
    * 비선형 토폴로지를 원한다.(ex: a residual connection(잔류 연결), a multi-branch model(다중 분기 모델))

## Sequential 모델 생성하기

In [6]:
model = keras.Sequential(
    [
        layers.Dense(2, activation = 'relu'),
        layers.Dense(3, activation = 'relu'),
        layers.Dense(4)
    ]
)

In [7]:
model.layers

#### add() 메서드를 통해 Sequential 모델을 점진적으로 작성할 수 있다.

In [8]:
model = keras.Sequential()
model.add(layers.Dense(2, activation = 'relu'))
model.add(layers.Dense(3, activation = 'relu'))
model.add(layers.Dense(4))

#### 레이어를 제거하는 pop() 메서드도 있다. Sequential모델은 레이어의 리스트와 동작이 매우 유사하다.

In [9]:
model.pop()
print(len(model.layers))

2


* Sequential 생성자는 Keras의 모든 레이어 또는 모델과 마찬가지로 name 인수를 허용한다.
* 이것은 의미론적으로 유의미한 이름으로 TensorBoard 그래프에 주석을 달 때 유용하다.

In [10]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## 미리 입력 형상 지정하기

In [11]:
layer = layers.Dense(3)
layer.weights  # Empty

[]

가중치는 모양이 입력의 형상에 따라 달라지기 때문에 입력에서 처음 호출될 때 가중치를 만든다.

In [12]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.71200293, -0.31786638,  0.2965542 ],
        [ 0.7543969 ,  0.18251765,  0.33326733],
        [-0.13233101, -0.08315754, -0.50664073],
        [ 0.29207087, -0.7730532 ,  0.01080489]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

- 당연히 이것은 Sequential 모델에도 적용된다. 
- 입력 형상이 없는 Sequential 모델을 인스턴스화할 때는 "빌드"되지 않는다. 
- 가중치가 없다(그리고 model.weights를 호출하면 오류가 발생함). 
- 모델에 처음 입력 데이터가 표시되면 가중치가 생성된다.

In [13]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6


In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (1, 2)                    10        
_________________________________________________________________
dense_8 (Dense)              (1, 3)                    9         
_________________________________________________________________
dense_9 (Dense)              (1, 4)                    16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


그러나 현재 출력 형상을 포함하여 지금까지 모델의 요약을 표시할 수 있도록 Sequential 모델을 점진적으로 빌드할 때 매우 유용할 수 있다. 이 경우 Input 객체를 모델에 전달하여 모델의 시작 형상을 알 수 있도록 모델을 시작해야 한다.

In [15]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.layers

간단한 대안은 첫 번째 레이어에 input_shape 인수를 전달하는 것이다.

In [17]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


이처럼 사전 정의된 입력 모양으로 빌드된 모델은 항상 가중치를 가지며(데이터를 보기 전에도) 항상 정의된 출력 형상을 갖는다.

일반적으로 Sequential 모델의 입력 형상을 알고 있는 경우 항상 Sequential 모델의 입력 형상을 지정하는 것이 좋다.

## 일반적인 디버깅 워크플로우: add() + summary()

새로운 Sequential 아키텍처를 구축할 때는 add() 하여 레이어를 점진적으로 쌓고 모델 요약을 자주 인쇄하는 것이 유용하다.

예를 들어 Conv2D 및 MaxPooling2D 레이어의 스택이 이미지 특성 맵을 다운 샘플링 하는 방법을 모니터링할 수 있다.

In [18]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
____________________________

## Sequential 모델을 사용한 특성 추출

In [19]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

다음은 한 레이어에서 특성만 추출하는 것과 유사한 예이다.

In [21]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)